In [ ]:
# ChatGPT
# a sequence of tokens, how they are sequenced, we give start of sequence the model continues
# Generatively Pre-trained Transformer
# Character level language model
# Tiny shakesphere - concatenated work of all shakesphere in text filter
# We model how charcters follow if given context
# Output is a shakesphere like text
# NanoGPT . train GPT with any text 124 million params - 38 hours training time in a single 8xA100 40GB Node
#  8xA100 40GB - One machine with 8 NVIDIA A100 GPUs, each with 40 GB of VRAM, connected via NVLink or PCIe for multi-GPU training
# using OpenWebText data 
# OpenwebText has 38GB of text data (40GB using SI units) from 8,013,769 documents
# train.py is 300 line training loop
# model.py a 300 line GPT model definition
# A multilayer perceptron is a fully connected feedforward net with at least three layers (input, hidden and output)
